### Connect to Drive

In [ ]:
#from google.colab import drive
#drive.mount('/gdrive')

In [ ]:
#%cd /gdrive/My Drive/ANN Exercises/Homework1

### Import libraries

In [1]:
!pip install -U tensorflow==2.10.0

import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2022-11-17 01:26:19.179722: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-17 01:26:19.349067: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2022-11-17 01:26:19.349113: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-17 01:26:19.374735: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to r

2.10.0


In [2]:
# Import dataset
#!unzip training_dataset_homework1.zip

# Dataset folders 
dataset_dir = '../input/homework1/training_data_final'

In [3]:
# Random seed for reproducibility
seed = 1337

random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
tf.compat.v1.set_random_seed(seed)

In [4]:
labels = ['Species1', # 1
          'Species2', # 2
          'Species3', # 3
          'Species4', # 4
          'Species5', # 5
          'Species6', # 6
          'Species7', # 7
          'Species8', # 8
]

In [5]:
# Images are divided into folders, one for each class. 
# If the images are organized in such a way, we can exploit the 
# ImageDataGenerator to read them from disk.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
def generate_new_class_samples(
    label_idx,                            # species to target as index of "labels" array
    output_dir=dataset_dir+'_augmented',
    n_images=1,                           # number of images to save
    one_aug=True,                         # perform one type of augmentation per image or multiple
    shuffle=False,                        # order of image retrieval
    hflip=False,                          # horizontal flip (True, False)
    vflip=False,                          # vertical flip(True, False)
    contrast=None,                        # contrast ([lower, upper])
    saturation=None,                      # saturation ([lower, upper]) 
    brightness=None,                      # brightness factor -1.0-1.0 ([lower, upper])
    rotation=None,                        # rotation factor as fraction of 2*PI ([lower, upper])
    rotation_fill_mode='nearest',         # fill mode for rotation {"constant", "reflect", "wrap", "nearest"})
    translation=None,                     # translation factors ([[height_lower, height_upper], [width_lower, width_upper]])
    translation_fill_mode='nearest',      # fill mode for translation {"constant", "reflect", "wrap", "nearest"})
    zoom=None,                            # zoom factors ([[height_lower, height_upper], [width_lower, width_upper]])
    zoom_fill_mode='nearest',             # fill mode for zoom {"constant", "reflect", "wrap", "nearest"})
    seed=None,
):

    def augment_all(image):
        if hflip:
            image = tf.image.random_flip_left_right(image, seed)
        if vflip:
            image = tf.image.random_flip_up_down(image, seed)
        if contrast and len(contrast) == 2:
            image = tf.image.random_contrast(image, lower=contrast[0], upper=contrast[1], seed=seed)
        if saturation and len(saturation) == 2:
            image = tf.image.random_saturation(image, lower=saturation[0], upper=saturation[1], seed=seed)
        if brightness:
            seq = tfk.Sequential([
                tfkl.RandomBrightness(factor=brightness, seed=seed)])
            image = seq(tf.expand_dims(image, 0), training=True)
        if rotation and rotation_fill_mode:
            seq = tfk.Sequential([
                tfkl.preprocessing.RandomRotation(rotation, fill_mode=rotation_fill_mode, seed=seed)])
            image = seq(tf.expand_dims(image, 0), training=True)
        if translation and translation_fill_mode:
            seq = tfk.Sequential([
                tfkl.experimental.preprocessing.RandomTranslation(
                    translation[0], translation[1], fill_mode=translation_fill_mode, seed=seed)])
            image = seq(tf.expand_dims(image, 0), training=True)
        if zoom and zoom_fill_mode:
            seq = tfk.Sequential([
                tfkl.experimental.preprocessing.RandomZoom(
                    height_factor=zoom[0], width_factor=zoom[1], fill_mode=zoom_fill_mode, seed=seed)])
            image = seq(tf.expand_dims(image, 0), training=True)
        return image
    
    def augment_one(image):
        images = []
        if hflip:
            images.append(tf.image.random_flip_left_right(image, seed))
        if vflip:
            images.append(tf.image.random_flip_up_down(image, seed))
        if contrast and len(contrast) == 2:
            images.append(tf.image.random_contrast(image, lower=contrast[0], upper=contrast[1], seed=seed))
        if saturation and len(saturation) == 2:
            images.append(tf.image.random_saturation(image, lower=saturation[0], upper=saturation[1], seed=seed))
        if brightness:
            seq = tfk.Sequential([
                tfkl.RandomBrightness(factor=brightness, seed=seed)])
            image = seq(tf.expand_dims(image, 0), training=True)
        if rotation and rotation_fill_mode:
            seq = tfk.Sequential([
                tfkl.experimental.preprocessing.RandomRotation(rotation, fill_mode=rotation_fill_mode, seed=seed)])
            image = seq(tf.expand_dims(image, 0), training=True)
        if translation and translation_fill_mode:
            seq = tfk.Sequential([
                tfkl.experimental.preprocessing.RandomTranslation(
                    translation[0], translation[1], fill_mode=translation_fill_mode, seed=seed)])
            image = seq(tf.expand_dims(image, 0), training=True)
        if zoom and zoom_fill_mode:
            seq = tfk.Sequential([
                tfkl.experimental.preprocessing.RandomZoom(
                    height_factor=zoom[0], width_factor=zoom[1], fill_mode=zoom_fill_mode, seed=seed)])
            image = seq(tf.expand_dims(image, 0), training=True)
        return images[random.randrange(len(images))]
    
    if one_aug:
        class_aug_data_gen = ImageDataGenerator(preprocessing_function=augment_one)
    else:
         class_aug_data_gen = ImageDataGenerator(preprocessing_function=augment_all)
    class_aug_gen = class_aug_data_gen.flow_from_directory(
        directory=dataset_dir,
        target_size=(96,96),
        color_mode='rgb',
        classes=[labels[label_idx]],
        class_mode='categorical',
        batch_size=1,
        shuffle=shuffle,
        save_to_dir=output_dir,
        save_prefix='aug',
        seed=seed,
    )
    
    for i, batch in enumerate(class_aug_gen):
        if i >= n_images-1:
            break 

In [9]:
# Generating extra sample for Species 1
generate_new_class_samples(
    0, 
    '/kaggle/working/', 
    one_aug=False, 
    #zoom=[[0.1, 0.2], 0], 
    brightness=[-0.3, 0.4],
    seed=seed,
)

Found 186 images belonging to 1 classes.


In [ ]:
# Create an instance of ImageDataGenerator for training, validation, and test sets
train_data_gen = ImageDataGenerator(
        validation_split=0.15, 
        rescale=1/255.,
)

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
train_gen = train_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=labels, # can be set to labels
                                               subset="training",
                                               class_mode='categorical',
                                               batch_size=64,
                                               shuffle=True,
                                               seed=seed)
valid_gen = train_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=labels, # can be set to labels
                                               subset="validation",
                                               class_mode='categorical',
                                               batch_size=64,
                                               shuffle=False,
                                               seed=seed)

In [ ]:
print("Assigned labels")
print(train_gen.class_indices)
print()
print("Target classes")
print(train_gen.classes)

In [ ]:
def get_next_batch(generator):
    batch = next(generator)

    image = batch[0]
    target = batch[1]
    
    print("(Input) image shape:", image.shape)
    print("Target shape:",target.shape)

    # Visualize only the first sample
    image = image[0]
    target = target[0]
    target_idx = np.argmax(target)
    print()
    print("Categorical label:", target)
    print("Label:", target_idx)
    print("Class name:", labels[target_idx])
    fig = plt.figure(figsize=(6, 4))
    
    image = image*255
    
    plt.imshow(np.uint8(image))

    return batch

In [ ]:
# Get a sample from dataset and show info
_ = get_next_batch(valid_gen)